In [1]:
# Run the top two cells in this notebook first, every time you reopen your notebook.
%load_ext autoreload
%autoreload 2

In [18]:
%matplotlib inline
# Some imports that may be helpful in later cells
import numpy as np
import matplotlib.pyplot as plt
import animal_guessing_game as game
import random 

### Animal Guessing Game

In the first few parts of this problem you'll be playing a guessing game. The computer chooses an animal at random, and your goal is to guess which animal has been chosen. You will first be shown two features of the animal and asked to make a guess. If you guess incorrectly, more features will be shown and you'll get more chances to guess the animal. Your score is the total number of tries it takes you to guess the animal. 

<div class="alert alert-success">Play five iterations of the guessing game by uncommenting and running the cell below; your results will be saved in the data folder to allow you to analyze them later. You should also submit this file when you upload your assignment. (Note: Some of these features are rather odd - do your best).
<p><p>
After running the whole game and checking that your data file is saved in the data folder, you may want to comment out this cell so that you can use "Run All" to run all subsequent cells and not have to play the game each time.</div>

In [40]:
trials = game.play_game("data/classes.txt","data/predicates.txt", "data/predicate-matrix-binary.txt")

----------------------------------------------------------------------
Features: ['plains', 'muscle']
Animals: ['dalmatian', 'ox', 'chihuahua', 'giant panda', 'mouse', 'rabbit', 'leopard', 'bobcat']


KeyboardInterrupt: 

<div class="alert alert-success">Briefly, describe your experience playing the game. Did it become more difficult with more animals? Do you feel like this difficulty scaled linearly (i.e., was it twice as hard to play with twice as many animals, or was it less than twice as hard, or more than twice as hard)?</div>

### Modeling the Animal Guessing Game: Rule-based categories

<div class="alert alert-success">Let's try to model the inferences made by players of this game using a simple program. Your model should play the game by looking at the given choices (animals) and observations (features) and ruling out possible hypotheses. Specifically, write a function <tt>guess(animal_choices, animal_features, animal_names, feature_names, feature_matrix)</tt>, where <tt>animal_choices</tt> and <tt>animal_features</tt> are the possibilities for the animal and the features that have been given to the model so far (animal_names, feature_names, and feature_matrix are the lists representing the data loaded from the text files) in <tt>animal_guessing_game.py</tt> which chooses uniformly at random one of the choices that matches all of the listed features. Uniformly at random means that all animals in <tt>animal_choices</tt> that have the listed features should be equally likely to be chosen.
<p>
In the cell below, test your <tt>guess</tt> function to make sure it behaves correctly.</div>

In [26]:
animal_names = game.load_name_data("data/classes.txt")
feature_names = game.load_name_data("data/predicates.txt")
feature_matrix = game.load_animal_feature_data("data/predicate-matrix-binary.txt")

def guess(animal_choices, animal_features, animal_names, feature_names, feature_matrix):
    animal_inds = [animal_names.index(anim) for anim in animal_choices] #find indecies for animals
    feature_inds = [feature_names.index(feat) for feat in animal_features] #find indecies for features
    guesses = [] 
    for a in animal_inds: #find all animals that have all features
         if all([feature_matrix[a][f] ==1 for f in feature_inds]): #if the animal has all of the features
                guesses.append(animal_names[a])
    return random.choice(guesses) #pick an animal at random that has all of the features


In [27]:
animal_choices = ['rat', 'skunk', 'walrus', 'cow', 'siamese cat', 'bat', 'fox', 'rabbit', 'lion']
animal_features = ['paws', 'smart']
guess(animal_choices, animal_features, animal_names, feature_names, feature_matrix)

'rat'

<div class="alert alert-success">Now, write a function <tt>model</tt> that will have your computer play 500 iterations of each of the five hypothesis sizes (2, 4, 8, 16, and 32). This function takes no parameters. You'll likely want to make a function like <tt>human_query</tt> for the computer model - e.g., <tt>computer_query</tt>. If you do this, you can use the <tt>run</tt> function that I've written to take care of much of the running of the game. One problem that may arise is what to pass in for stream, as you don't really want to print out all 500*5 games that the computer plays. Instead, you can omit writing out these games by opening a stream like this:
<br>
<tt>stream = open(os.devnull, 'w')</tt>
</br>

Pass this stream to the run function, and you won't see any output.
<p>
<tt>model</tt> does not need to return anything, but it should write out a data file with the model results to "data/model_trial_data.txt". This file should have 500 lines, each with 5 comma-separated numbers that correspond to the number of guesses for each hypothesis size. You're welcome to use the <tt>save_trial_data</tt> function that I wrote to help.
<p><p>
In the cell below, call your <tt>model</tt> function so that it runs and saves the data.
<div>

In [65]:
def setUpGame(h_size, animal_names, feature_names, feature_matrix):
    feature_names = np.array(feature_names)
    animals = random.sample(animal_names, h_size)
    answer = animals[0] #first animal is the answer
    f_indexes = np.array([f == 1 for f in feature_matrix[animal_names.index(answer)]])
    features = feature_names[f_indexes]
    return(animals, features)

def runGame(h_size, animal_names, feature_names, feature_matrix):
    results = []
    for trial in range(500):
        animals, features = setUpGame(h_size, animal_names, feature_names, feature_matrix)
        #features = features.shuffle() #to make it random
        g = "wrong"
        tries = 0
        correct = animals[0]
        print(correct)
        while g != correct:
            ### 
            # To save time in computation, we will filter the animals so that animals
            # filtered in a previus feature list are still filtered out
            ##
            animal_inds = [animal_names.index(anim) for anim in animals] #find indicies for animals
            print(animal_inds)
            feature_inds = [feature_names.index(feat) for feat in features] #find indicies for features
            print(feature_inds)
            valid = []
            for a in animal_inds: #find all animals that have all features
                 if all([feature_matrix[a][f] ==1 for f in feature_inds]): #if the animal has all of the features
                    valid.append(animal_names[a])
            print(animals)
            animals = valid
            print(animals)
            #guess randomly
            g = random.sample(animals,1)[0]
            #print(g)
            #print(animals)
            animals.remove(g) #don't guess the same animal twice 
            tries +=1
            print(g)
        print()
        print()
        results.append(tries)
    return(results)
    
def model():
    animal_names = game.load_name_data("data/classes.txt")
    feature_names = game.load_name_data("data/predicates.txt")
    feature_matrix = game.load_animal_feature_data("data/predicate-matrix-binary.txt")
    #two = runGame(2, animal_names, feature_names, feature_matrix)
    print('two')
    #print(two)
    four = runGame(4, animal_names, feature_names, feature_matrix)
    print('four')
    print(four)
    eight = runGame(8, animal_names, feature_names, feature_matrix)
    print('eight')
    print(eight)
    sixteen = runGame(16, animal_names, feature_names, feature_matrix)
    print('sixteen')
    print(sixteen)
    thirtytwo = runGame(32, animal_names, feature_names, feature_matrix)
    print('thirtytwo')
    print(thirtytwo)
    
        
            
    
model()

two
hamster
[25, 26, 14, 44]
[0, 1, 3, 4, 8, 11, 15, 16, 22, 25, 26, 28, 31, 33, 35, 37, 38, 39, 42, 45, 46, 47, 48, 49, 50, 54, 56, 57, 62, 70, 74, 79, 82, 83, 84]
['hamster', 'squirrel', 'leopard', 'polar bear']
['hamster']
hamster


raccoon
[47, 11, 16, 25]
[0, 1, 4, 8, 9, 10, 11, 15, 21, 22, 25, 26, 27, 31, 38, 39, 45, 46, 48, 49, 50, 51, 52, 54, 56, 59, 62, 63, 69, 70, 72, 74, 76, 78, 80, 82, 83]
['raccoon', 'mole', 'spider monkey', 'hamster']
['raccoon']
raccoon


otter
[35, 40, 10, 22]
[0, 3, 11, 15, 16, 17, 18, 22, 25, 26, 27, 28, 31, 36, 39, 45, 46, 50, 51, 58, 62, 63, 64, 65, 69, 73, 75, 79, 80, 82, 83]
['otter', 'bobcat', 'skunk', 'sheep']
['otter']
otter


blue whale
[8, 0, 9, 14]
[2, 4, 9, 12, 13, 14, 16, 18, 25, 29, 36, 39, 40, 41, 43, 47, 51, 53, 60, 62, 63, 64, 73, 75, 79, 80, 81, 82]
['blue whale', 'antelope', 'siamese cat', 'leopard']
['blue whale']
blue whale


dolphin
[49, 8, 14, 46]
[1, 2, 4, 12, 13, 14, 17, 18, 25, 26, 36, 39, 41, 43, 46, 50, 51, 62, 63, 65, 73, 7

[3, 5, 7, 9, 11, 15, 17, 21, 22, 25, 27, 31, 38, 39, 41, 43, 45, 46, 49, 50, 52, 56, 58, 61, 62, 63, 68, 69, 72, 74, 76, 77, 78, 80, 82]
['bobcat', 'squirrel', 'siamese cat', 'horse']
['bobcat']
bobcat


otter
[35, 29, 49, 34]
[0, 3, 11, 15, 16, 17, 18, 22, 25, 26, 27, 28, 31, 36, 39, 45, 46, 50, 51, 58, 62, 63, 64, 65, 69, 73, 75, 79, 80, 82, 83]
['otter', 'bat', 'dolphin', 'weasel']
['otter']
otter


weasel
[34, 41, 23, 1]
[0, 3, 4, 11, 15, 17, 22, 25, 26, 27, 31, 33, 37, 38, 39, 43, 45, 46, 48, 50, 52, 56, 58, 62, 63, 69, 70, 74, 78, 80, 82]
['weasel', 'pig', 'seal', 'grizzly bear']
['weasel']
weasel


cow
[48, 46, 32, 42]
[0, 1, 3, 8, 9, 11, 13, 14, 16, 20, 25, 26, 30, 33, 38, 40, 41, 45, 46, 47, 54, 57, 62, 63, 68, 70, 74, 79, 81, 84]
['cow', 'walrus', 'chihuahua', 'lion']
['cow']
cow


dalmatian
[4, 17, 44, 9]
[0, 1, 8, 9, 11, 12, 14, 17, 22, 23, 25, 26, 27, 38, 39, 41, 43, 45, 46, 50, 52, 62, 63, 74, 79, 80, 81, 82, 84]
['dalmatian', 'humpback whale', 'polar bear', 'siamese cat'

[1, 11, 13, 14, 16, 21, 22, 26, 27, 31, 33, 36, 38, 39, 40, 41, 44, 45, 46, 47, 49, 50, 51, 52, 56, 58, 59, 61, 62, 63, 64, 65, 73, 74, 75, 78, 82]
['polar bear', 'humpback whale', 'sheep', 'lion']
['polar bear']
polar bear


rabbit
[28, 39, 21, 23]
[0, 1, 3, 4, 8, 11, 15, 16, 22, 25, 26, 28, 35, 39, 42, 45, 46, 47, 50, 54, 56, 57, 62, 63, 67, 68, 69, 70, 74, 79, 81, 83, 84]
['rabbit', 'deer', 'fox', 'seal']
['rabbit']
rabbit


siamese cat
[9, 39, 30, 21]
[0, 1, 3, 4, 8, 11, 15, 17, 21, 22, 23, 25, 26, 27, 31, 38, 39, 42, 43, 45, 46, 47, 48, 50, 51, 52, 58, 61, 62, 63, 74, 78, 79, 80, 82, 84]
['siamese cat', 'deer', 'giraffe', 'fox']
['siamese cat']
siamese cat


rhinoceros
[27, 49, 14, 2]
[4, 12, 13, 14, 16, 20, 30, 32, 33, 38, 40, 41, 45, 47, 54, 57, 63, 67, 71, 74, 78, 81, 82]
['rhinoceros', 'dolphin', 'leopard', 'killer whale']
['rhinoceros']
rhinoceros


buffalo
[36, 34, 0, 47]
[0, 3, 11, 13, 14, 16, 20, 26, 30, 33, 38, 39, 40, 41, 43, 45, 47, 54, 56, 57, 62, 68, 74, 78, 81]
['buf

walrus


skunk
[10, 12, 11, 46]
[0, 1, 10, 11, 15, 16, 21, 22, 25, 33, 38, 39, 40, 42, 45, 46, 48, 49, 54, 56, 62, 63, 69, 70, 74, 79, 82]
['skunk', 'tiger', 'mole', 'walrus']
['skunk']
skunk


sheep
[22, 25, 10, 17]
[0, 1, 11, 16, 20, 26, 33, 38, 40, 42, 45, 47, 54, 57, 62, 63, 68, 70, 72, 74, 79, 81, 84]
['sheep', 'hamster', 'skunk', 'humpback whale']
['sheep']
sheep


moose
[15, 1, 22, 34]
[3, 11, 13, 14, 16, 20, 23, 24, 25, 26, 30, 33, 38, 39, 40, 41, 43, 45, 47, 54, 56, 57, 62, 63, 64, 68, 69, 70, 72, 74, 79, 81, 82]
['moose', 'grizzly bear', 'sheep', 'weasel']
['moose']
moose


deer
[39, 33, 9, 26]
[3, 8, 9, 11, 14, 17, 20, 23, 24, 25, 26, 30, 38, 39, 41, 43, 45, 46, 50, 54, 56, 57, 62, 63, 68, 69, 70, 72, 74, 79, 80, 81, 83]
['deer', 'rat', 'siamese cat', 'squirrel']
['deer']
deer


hippopotamus
[13, 29, 18, 42]
[4, 12, 13, 14, 16, 26, 29, 36, 38, 40, 41, 43, 45, 47, 51, 54, 63, 71, 74, 75, 79, 82]
['hippopotamus', 'bat', 'elephant', 'lion']
['hippopotamus']
hippopotamus


zebra

[43, 17, 34, 19, 0, 9, 40, 31]
[1, 3, 4, 11, 15, 22, 25, 26, 28, 33, 37, 38, 39, 42, 45, 46, 48, 49, 50, 54, 57, 59, 62, 63, 68, 69, 70, 74, 79, 81, 83, 84]
['mouse', 'humpback whale', 'weasel', 'gorilla', 'antelope', 'siamese cat', 'bobcat', 'wolf']
['mouse']
mouse


chihuahua
[32, 40, 37, 9, 10, 2, 14, 36]
[0, 3, 4, 11, 15, 17, 21, 22, 25, 27, 31, 33, 38, 39, 42, 45, 46, 52, 62, 63, 74, 78, 79, 80, 82, 84]
['chihuahua', 'bobcat', 'zebra', 'siamese cat', 'skunk', 'killer whale', 'leopard', 'buffalo']
['chihuahua']
chihuahua


fox
[21, 9, 20, 42, 25, 37, 39, 16]
[3, 5, 6, 11, 15, 17, 21, 22, 25, 26, 27, 31, 38, 39, 41, 43, 45, 46, 48, 49, 50, 51, 52, 56, 58, 61, 62, 63, 68, 69, 70, 74, 78, 80, 82, 83]
['fox', 'siamese cat', 'ox', 'lion', 'hamster', 'zebra', 'deer', 'spider monkey']
['fox']
fox


bat
[29, 49, 31, 26, 13, 0, 7, 27]
[0, 3, 4, 11, 12, 13, 15, 17, 26, 27, 31, 33, 34, 39, 42, 43, 44, 46, 47, 48, 49, 50, 52, 54, 55, 56, 58, 59, 62, 63, 69, 71, 72, 76, 77, 78, 80, 81, 83]
['ba

...

['chimpanzee']
chimpanzee


tiger
[12, 45, 8, 2, 9, 38, 43, 5, 27, 41, 3, 15, 35, 20, 13, 23]
[0, 1, 5, 10, 11, 14, 17, 21, 22, 25, 26, 27, 28, 31, 33, 38, 39, 41, 43, 45, 46, 48, 50, 52, 58, 61, 63, 67, 68, 69, 71, 74, 78, 80, 81, 82, 83]
['tiger', 'collie', 'blue whale', 'killer whale', 'siamese cat', 'giant panda', 'mouse', 'persian cat', 'rhinoceros', 'pig', 'beaver', 'moose', 'otter', 'ox', 'hippopotamus', 'seal']
['tiger']
tiger


giant panda
[38, 24, 47, 32, 43, 28, 22, 30, 33, 31, 12, 48, 19, 42, 27, 35]
[0, 1, 8, 9, 11, 14, 16, 21, 22, 26, 28, 31, 38, 40, 41, 44, 45, 47, 51, 54, 56, 57, 62, 63, 67, 69, 71, 74, 76, 79, 80, 81, 82, 83, 84]
['giant panda', 'chimpanzee', 'raccoon', 'chihuahua', 'mouse', 'rabbit', 'sheep', 'giraffe', 'rat', 'wolf', 'tiger', 'cow', 'gorilla', 'lion', 'rhinoceros', 'otter']
['giant panda']
giant panda


chihuahua
[32, 45, 44, 46, 48, 10, 28, 14, 49, 25, 37, 43, 15, 2, 38, 4]
[0, 3, 4, 11, 15, 17, 21, 22, 25, 27, 31, 33, 38, 39, 42, 45, 46, 52, 62, 63

rabbit


collie
[45, 49, 36, 5, 9, 22, 23, 38, 18, 2, 4, 25, 42, 26, 35, 43]
[1, 3, 8, 11, 14, 15, 17, 22, 25, 26, 27, 38, 39, 45, 46, 50, 52, 58, 62, 63, 74, 79, 80, 82, 84]
['collie', 'dolphin', 'buffalo', 'persian cat', 'siamese cat', 'sheep', 'seal', 'giant panda', 'elephant', 'killer whale', 'dalmatian', 'hamster', 'lion', 'squirrel', 'otter', 'mouse']
['collie']
collie


humpback whale
[17, 4, 5, 18, 21, 39, 1, 23, 8, 38, 49, 48, 27, 45, 47, 12]
[0, 2, 4, 12, 13, 14, 16, 18, 25, 29, 36, 39, 40, 41, 43, 47, 51, 53, 60, 62, 63, 64, 65, 73, 75, 79, 80, 81]
['humpback whale', 'dalmatian', 'persian cat', 'elephant', 'fox', 'deer', 'grizzly bear', 'seal', 'blue whale', 'giant panda', 'dolphin', 'cow', 'rhinoceros', 'collie', 'raccoon', 'tiger']
['humpback whale']
humpback whale


polar bear
[44, 43, 14, 45, 4, 28, 26, 11, 20, 38, 19, 1, 6, 0, 25, 39]
[1, 11, 13, 14, 16, 21, 22, 26, 27, 31, 33, 36, 38, 39, 40, 41, 44, 45, 46, 47, 49, 50, 51, 52, 56, 58, 59, 61, 62, 63, 64, 65, 73, 74, 7

['seal', 'buffalo', 'walrus', 'bobcat', 'rhinoceros', 'wolf', 'humpback whale', 'siamese cat', 'persian cat', 'giraffe', 'mouse', 'dolphin', 'deer', 'hamster', 'moose', 'elephant', 'antelope', 'hippopotamus', 'gorilla', 'dalmatian', 'rabbit', 'tiger', 'polar bear', 'collie', 'pig', 'squirrel', 'fox', 'chihuahua', 'horse', 'giant panda', 'rat', 'bat']
['seal']
seal


lion
[42, 23, 19, 20, 16, 48, 39, 11, 38, 0, 6, 26, 47, 7, 15, 28, 27, 45, 4, 9, 40, 43, 8, 21, 30, 17, 5, 36, 18, 12, 24, 13]
[3, 7, 11, 14, 16, 17, 21, 22, 25, 27, 31, 33, 38, 39, 41, 43, 45, 46, 47, 50, 52, 56, 58, 61, 63, 66, 67, 69, 71, 72, 74, 78, 80, 81, 83]
['lion', 'seal', 'gorilla', 'ox', 'spider monkey', 'cow', 'deer', 'mole', 'giant panda', 'antelope', 'horse', 'squirrel', 'raccoon', 'german shepherd', 'moose', 'rabbit', 'rhinoceros', 'collie', 'dalmatian', 'siamese cat', 'bobcat', 'mouse', 'blue whale', 'fox', 'giraffe', 'humpback whale', 'persian cat', 'buffalo', 'elephant', 'tiger', 'chimpanzee', 'hippopotamu


german shepherd
[7, 19, 47, 30, 32, 4, 48, 28, 42, 37, 17, 23, 12, 22, 21, 44, 14, 38, 34, 24, 43, 13, 33, 39, 3, 11, 18, 29, 26, 20, 1, 8]
[0, 3, 4, 8, 11, 14, 17, 21, 22, 23, 25, 26, 27, 31, 33, 38, 39, 41, 43, 45, 46, 50, 52, 58, 61, 62, 63, 68, 74, 78, 80, 82, 84]
['german shepherd', 'gorilla', 'raccoon', 'giraffe', 'chihuahua', 'dalmatian', 'cow', 'rabbit', 'lion', 'zebra', 'humpback whale', 'seal', 'tiger', 'sheep', 'fox', 'polar bear', 'leopard', 'giant panda', 'weasel', 'chimpanzee', 'mouse', 'hippopotamus', 'rat', 'deer', 'beaver', 'mole', 'elephant', 'bat', 'squirrel', 'ox', 'grizzly bear', 'blue whale']
['german shepherd']
german shepherd


sheep
[22, 13, 39, 46, 1, 18, 33, 10, 23, 30, 28, 19, 15, 0, 32, 43, 20, 25, 2, 9, 16, 21, 36, 3, 41, 42, 29, 26, 5, 17, 8, 45]
[0, 1, 11, 16, 20, 26, 33, 38, 40, 42, 45, 47, 54, 57, 62, 63, 68, 70, 72, 74, 79, 81, 84]
['sheep', 'hippopotamus', 'deer', 'walrus', 'grizzly bear', 'elephant', 'rat', 'skunk', 'seal', 'giraffe', 'rabbit', 'go

skunk
[10, 43, 7, 22, 21, 28, 37, 42, 3, 9, 19, 33, 5, 26, 35, 47, 32, 29, 45, 49, 34, 4, 14, 13, 44, 20, 27, 17, 1, 8, 48, 24]
[0, 1, 10, 11, 15, 16, 21, 22, 25, 33, 38, 39, 40, 42, 45, 46, 48, 49, 54, 56, 62, 63, 69, 70, 74, 79, 82]
['skunk', 'mouse', 'german shepherd', 'sheep', 'fox', 'rabbit', 'zebra', 'lion', 'beaver', 'siamese cat', 'gorilla', 'rat', 'persian cat', 'squirrel', 'otter', 'raccoon', 'chihuahua', 'bat', 'collie', 'dolphin', 'weasel', 'dalmatian', 'leopard', 'hippopotamus', 'polar bear', 'ox', 'rhinoceros', 'humpback whale', 'grizzly bear', 'blue whale', 'cow', 'chimpanzee']
['skunk']
skunk


gorilla
[19, 17, 41, 22, 16, 21, 39, 31, 28, 2, 49, 25, 27, 0, 44, 48, 33, 43, 35, 8, 24, 1, 23, 37, 12, 5, 4, 13, 46, 32, 3, 30]
[0, 3, 11, 13, 14, 16, 19, 23, 26, 27, 33, 38, 39, 41, 43, 44, 45, 46, 50, 52, 54, 56, 63, 69, 71, 74, 76, 78, 80, 81, 83]
['gorilla', 'humpback whale', 'pig', 'sheep', 'spider monkey', 'fox', 'deer', 'wolf', 'rabbit', 'killer whale', 'dolphin', 'hamst

### Modeling the Animal Guessing-Game: Knowledgeless Play

<div class="alert alert-success">The model above assumes the player knows all the animal-feature pairings. Imagine instead that the player knows only a subset of the feature pairings. We'll model this by assuming that we have a new player for each set of hypothesis sizes, and that player knows about <tt>n</tt> of the 85 features, where <tt>n</tt> is a parameter of the model. <tt>n</tt> random features are chosen, and when the model makes its guesses, it chooses a guess uniformly at random from those options that are consistent with the observed features that it knows about. If it doesn't know about an observed feature, it just ignores it. For example, if the model knows only about "small", and the observed features are "small" and "strainteeth", the model will choose an animal from the available choices that are small. Write a function <tt>bounded_model</tt> that takes in the argument <tt>n</tt> and plays 500 iterations of each of the 5 hypothesis sizes. It should write it's results to a file name "data/bounded_model_trial_data_n.txt", where the <tt>n</tt> is replaced by the value of the parameter <tt>n</tt>. Note that you can likely use a similar structure to the previous part - look at what parameters are taken in by the <tt>human_query</tt> function.
<p>
<p>
In the cell below, run your model with n = 5 and then with n = 20.
<div>

### Analyzing the Results

<div class="alert alert-success">In the cell below, write code to graph the trial data for each of your models as well as trial data that I've included from 294 people who played this game previously (stored in "data/old_trial_data.txt"). Make a line graph with separate lines for each model and for the people, where the x-axis represents the size of the hypothesis set and the y-axis represents the number of guesses. Plot the average across trials - e.g., while 294 people played the game, you'll have just one line for people and the y-value for hypothesis size 2 will the be average number of guesses people required when they had a hypothesis set of size 2. As for all graphs, please make sure your graph is appropriately labeled.

</div>

<div class="alert alert-success">How do the performances of the models compare to one another and to people? Are there systematic differences between each set of data for how the number of guesses scales with the number of animal choices? Explain your answers. Which model do you think provides the closest match to people? The furthest? For the model that is closest, explain whether you think this model is a good account of people's inductive inferences in this type of guessing games? For the model that is furthest, why do you think this model behaves differently from people? (I.e., what is missing in the model or what assumptions are incorrect that cause it to be a poor account of people's behavior?)
</div>

### Finding Rule-based Categories

(Definitions) One of the attractive features of the logical perspective on the world is that it seems to make categorization easy: we can just specify a definition of a category (e.g. "an even number is an integer that can be divided by 2 with no remainder") and check if something belongs to the category by checking if it satisfies the rule. In the next set of questions, you'll use the same dataset as in the previous parts to explore this idea.


<div class="alert alert-success">
In <tt>animal_guessing_game.py</tt>, write a function <tt>find_simplest_rule(target_animal, animal_names, features_names, feature_matrix)</tt> which takes as parameters an animal name, a list of animal names, a list of feature names, and a binary 2-dimensional list of animal-feature pairs (the feature matrix); as in previous parts, each inner list is one animal. This function should find the simplest conjunctive rule for identifying the given animal. A rule identifies an animal if it is true for the target animal and false for all other animals. The rule may include conjunctions of features or their negations. For example, the rule "lean and not strainteeth" would include all animals that have the feature lean and do not have the feature strainteeth. Your function should return a dictionary that has two keys: positive_features, where the value is a list of the names of features that are not negated; and negative_features, where the value is a list of the names of features that are negated. For example, the rule above would be represented by the following dictionary: <tt>{"positive_features" : ["lean"], "negative_features" : ["strainteeth"]}</tt>. The dictionary should return the simplest rule that identifies the target animal. The simplest rule is defined as the one that contains the fewest features (whether or not they are negated).
</div>

<div class="alert alert-success">
In the cell below, include some tests to make sure your function works. At the end, print out the best rule for identifying "elephant" if you give the function all possible features but only include the first 25 animals (that is, the length of animal_names should be 25 and the length of feature matrix should also be 25).
</div>

<div class="alert alert-success">
In the cell below, print out all animals in the full dataset of 50 animals that are consistent with the "elephant" rule you created above.
</div>

<div class="alert alert-success">
You may have found that your original rule did not work for the complete dataset, although you could use your function to find a new rule that does work for all 50 animals. If you were to refine your rule with the full dataset, do you think that the new rule would be consistent with all animals you might come in contact with in the world (both elephants and not)? Would continually refining your rule as you come into contact with more animals be an effective way to maintain a correct logical definition of each animal category? Explain your answers and discuss how you think this relates to whether logical definitions are a good model of categorization.
</div>


<div class="alert alert-success">
Assume people do maintain logical definitions of categories. Your function above finds the simplest rule for a particular animal. How does this run time of this function scale with the number of possible features and the number of possible animals? Do you think that the way this scales is a reasonable approximation for how people might form logical categories? Explain why or why not.
</div>